<a href="https://colab.research.google.com/github/abisubramanya27/CS6910_Assignment2/blob/main/partA/src/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mounting drive to store dataset
# from google.colab import drive

# drive.mount('/content/gdrive')
#!rm -rf inaturalist_12K

In [2]:
%%time
# %cd gdrive/MyDrive/assignments/cs6910/A2/Data
# !pwd
#!gdown --id 11SGStqp8Vug2GDzSpJDwQYHThLIjZFQn

!unzip -q inaturalist_12K.zip
!ls

# import zipfile
# import concurrent.futures

# zf = zipfile.ZipFile('inaturalist_12K.zip')

# def unzip(file):
#     zf.extract(file)

# with concurrent.futures.ProcessPoolExecutor() as executor:
#     executor.map(unzip, zf.infolist())

inaturalist_12K  inaturalist_12K.zip  sample_data  wandb
CPU times: user 888 ms, sys: 118 ms, total: 1.01 s
Wall time: 2min 56s


In [3]:
# !pip install split-folders

In [4]:
# import splitfolders

# # Splitting the training data into training and validation set
# splitfolders.ratio('./inaturalist_12K/train', output='./inaturalist_12K/output', seed=1337, ratio=(.9, .1), group_prefix=None)

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping

physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


In [6]:
def build_model_partA(inp_img_shape, K_list, F_list, no_neurons_dense, no_classes = 10, activation_fn_list = ['relu']*6, 
                      P_list = ['valid']*10, S_list = [1]*10, BN_yes = False, dropout_p = 0):
    '''
    Function to build the model comprising (5 conv+relu+maxpooling layers + 1 dense FC layer) for part A in keras
    Arguments :
        inp_img_shape -- shape of input image
        K_list -- List of number of filters in each non FC layer
        F_list -- List of size of filters (assumed same dimension in width and height) in each non FC layer  
        no_neurons_dense -- Number of neurons in the dense FC layer
        no_classes -- Number of output classes in the classification problem
        activation_fn_list -- List of activation function in each convolution and FC layer
        P_list -- List of padding options in each non FC layer 
                  ('valid' : no padding, 'same' : padding to make input and output same dimensions)
        S_list -- List of strides (assumed equal in width and height) in each non FC layer
        BN_yes -- True : Batch normalisation (BN) should be used, False : BN should not be used
        dropout_p -- Probability of dropping out a neuron
                     (The dropout is added for the single dense hidden layer alone after referring to many CNN architecture papers)

    Returns :
        model -- The keras sequential model of the CNN created
    '''
    model = Sequential()
    # First layer
    model.add(Conv2D(filters = K_list[0], kernel_size = (F_list[0], F_list[0]), strides = (S_list[0], S_list[0]), 
                     padding = P_list[0], input_shape = inp_img_shape))
    if BN_yes:
        model.add(BatchNormalization())
    model.add(Activation(activation_fn_list[0]))
    model.add(MaxPooling2D(pool_size=(F_list[1], F_list[1]), strides = (S_list[1], S_list[1]), padding = P_list[1]))

    # 4 Conv-relu-MaxPooling layers
    for l in range(1, 5):
        model.add(Conv2D(filters = K_list[2*l], kernel_size = (F_list[2*l], F_list[2*l]), strides = (S_list[2*l], S_list[2*l]), 
                         padding = P_list[2*l]))
        if BN_yes:
            model.add(BatchNormalization())
        model.add(Activation(activation_fn_list[l]))
        model.add(MaxPooling2D(pool_size = (F_list[2*l+1], F_list[2*l+1]), strides = (S_list[2*l+1], S_list[2*l+1]), padding = P_list[2*l+1]))
    
    # 1 dense FC layer
    model.add(Flatten())
    model.add(Dropout(dropout_p))
    model.add(Dense(units = no_neurons_dense))
    if BN_yes:
        model.add(BatchNormalization())
    model.add(Activation(activation = activation_fn_list[5]))

    # Output layer
    model.add(Dense(units = no_classes))
    if BN_yes:
        model.add(BatchNormalization())
    model.add(Activation(activation = 'softmax'))

    return model
    

In [7]:
!pip install --upgrade wandb
!wandb login 6746f968d95eb71e281d6c7772a0469574430408

Requirement already up-to-date: wandb in /usr/local/lib/python3.7/dist-packages (0.10.24)
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
from tensorflow.keras import layers

# Model for resizing and rescaling images
image_rescale = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255)
])

# Model for performing random transformations for data augmentation
data_augmentation = Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal"),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomTranslation(0.2, 0.2),
    layers.experimental.preprocessing.RandomZoom(0.2, 0.2),
    layers.experimental.preprocessing.RandomContrast(0.2)
])

def prepare_data(data_path, inp_img_shape, batch_size, img_preprocess, data_augmentation, data_augment_yes = False, shuffle = True):
    AUTOTUNE = tf.data.AUTOTUNE
    dataset = image_dataset_from_directory(
        data_path, labels='inferred', color_mode='rgb', batch_size=batch_size, image_size=inp_img_shape[:-1], shuffle=shuffle,
        label_mode = 'categorical'
    )
    
    dataset = dataset.map(lambda x, y: (img_preprocess(x), y), num_parallel_calls=AUTOTUNE)

    # Use data augmentation only if data_augment_yes == True (Training set only requires data augmentation)
    if data_augment_yes:
        dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)

    # Use buffered prefecting on datasets
    return dataset.prefetch(buffer_size=AUTOTUNE)


def data_generator(train_data_path, inp_img_shape, batch_size, data_augment_yes = False, val_data_path = None, test_data_path = None):
    train_data = prepare_data(train_data_path, inp_img_shape, batch_size, image_rescale, data_augmentation, data_augment_yes)
    val_data = None
    if val_data_path is not None:
        val_data = prepare_data(val_data_path, inp_img_shape, batch_size, image_rescale, data_augmentation, False)
    test_data = None
    if test_data_path is not None:
        test_data = prepare_data(test_data_path, inp_img_shape, batch_size, image_rescale, data_augmentation, False)
    
    return train_data, val_data, test_data


In [9]:
# def data_generator(train_data_path, inp_img_shape, batch_size, data_augment_yes = False, val_data_path = None, test_data_path = None):
#     # Techniques for data augmentation sent to ImageDataGenerator 
#     data_augment_params = {
#         'rotation_range': 25,
#         'height_shift_range': 0.2,
#         'width_shift_range': 0.2,
#         'channel_shift_range': 40,
#         'brightness_range': (0.2, 0.7),
#         'zoom_range': 0.2,
#         'horizontal_flip': True 
#     }

#     train_gen_param = data_augment_params if data_augment_yes else dict()

#     # Generators for training, validation and test set image data for Part-A from the respective 
#     train_generator = ImageDataGenerator(rescale = 1./255, **train_gen_param).flow_from_directory(train_data_path, 
#                                                                                                   target_size = inp_img_shape[:-1], 
#                                                                                                   batch_size = batch_size, 
#                                                                                                   class_mode = 'categorical')
#     val_generator = None
#     if val_data_path is not None:
#         val_generator = ImageDataGenerator(rescale = 1./255).flow_from_directory(val_data_path, target_size = inp_img_shape[:-1], 
#                                                                                  batch_size = batch_size, class_mode = 'categorical')  
#     test_generator = None
#     if test_data_path is not None:
#         test_generator = ImageDataGenerator(rescale = 1./255).flow_from_directory(test_data_path, target_size = inp_img_shape[:-1], 
#                                                                                   batch_size = batch_size, class_mode = 'categorical')
    
#     return train_generator, val_generator, test_generator


In [10]:
import wandb
from wandb.keras import WandbCallback

config_1 = {
    "learning_rate": 5e-3,
    "epochs": 20,
    "batch_size": 64,
    "loss_function": 'categorical_crossentropy',
    "architecture": 'CNN',
    "dataset": "iNaturalist_12K"
}

In [11]:
import math

def train_model(model, train_data, config, val_data = None):
    model.compile(optimizer = Adam(learning_rate=config['learning_rate']), loss = config['loss_function'], metrics = ['accuracy'])
    model.fit(train_data,
              epochs = config['epochs'], 
              validation_data = val_data,
              verbose = 2,
              callbacks = [WandbCallback()])
    
    return model

def get_klist(method, start1):
  if method == 'same':
    return [start] * 10
  elif method == 'double':
    start = start1
    vals = []
    for i in range(5):
      vals.append(start)
      vals.append(start)
      start *= 2
    return vals
  else:
    start = start1
    vals = []
    for i in range(5):
      vals.append(start)
      vals.append(start)
      start = max(start//2, 1)
    return vals

In [13]:
# Running sample run

def CNN(inp_img_shape, train_data_path, K_list, F_list, no_neurons_dense, config, no_classes = 10, activation_fn_list = ['relu']*6, 
        P_list = ['valid']*10, S_list = [1]*10, BN_yes = False, dropout_p = 0, val_data_path = None, test_data_path = None, 
        data_augment_yes = False):
    
    #run = wandb.init(project="assignment2", entity="abisheks", reinit=True, config=config)
    tf.keras.backend.clear_session()

    model = build_model_partA(inp_img_shape, K_list, F_list, no_neurons_dense, no_classes, activation_fn_list, P_list, S_list, BN_yes, dropout_p)
    # model.summary()
    train_gen, val_gen, test_gen = data_generator(train_data_path, inp_img_shape, config['batch_size'], data_augment_yes, 
                                                  val_data_path, test_data_path)
    model = train_model(model, train_gen, config, val_gen)
    
    #run.finish()

    return model

# Hyperparameters for building the model for Part-A
K_list_1 = [32, 32, 32, 32, 64, 64, 64, 64, 128, 128]       # List of number of filters in each non FC layer
F_list_1 = [11, 3, 5, 3, 3, 3, 3, 3, 3, 3]                  # List of size of filters in each non FC layer  
no_neurons_dense_1 = 256                                    # Number of neurons in the dense FC layer
activation_fn_list_1 = ['relu']*6                           # List of activation function in each convolution and FC layer
P_list_1 = ['valid']*10                                     # List of padding options in each non FC layer ('valid' : no padding, 'same' : padding to make input and output same dimensions)
S_list_1 = [4, 2, 1, 1, 1, 1, 1, 2, 1, 1]                   # List of number of strides in each non FC layer
inp_img_shape_1 = (227, 227, 3)                             # Shape of input image from data
no_classes_1 = 10                                           # Number of output classes in the classification problem
BN_1 = True                                                 # True : Batch normalisation (BN) should be used, False : BN should not be used
dropout_p_1 = 0.3                                           # Probability of dropping out a neuron


# PART-A, Question 1 -- Building a model with (5 conv+relu+maxpooling layers + 1 dense FC layer) for image classification objective 
# = CNN(inp_img_shape_1, './inaturalist_12K/train', K_list_1, F_list_1, no_neurons_dense_1, config_1, no_classes_1, 
#             activation_fn_list_1, P_list_1, S_list_1, BN_1, dropout_p_1, './inaturalist_12K/val', './inaturalist_12K/test', False)

In [15]:
sweep_config = {
    'name': 'CNN',
    'method': 'bayes',                   # Possible search : grid, random, bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'no_filters': {
            'values': [32, 64]
        },
        'filter_organization': {
            'values': ['same', 'double', 'half']
        },
        'data_augmented': {
            'values': [True, False]
        },
        'dropout' :{
            'values': [0.2, 0.3]
        },
        'batch_normalization': {
            'values': [True, False]
        }
    }
}

In [16]:
def sweep_wrapper(data_path = './inaturalist_12K'):
  
  # Wrapper function to call the CNN function for sweeping with different hyperparameters
  # loss - (string) Loss function used. Takes values only in ['cross-entropy', 'squared-error']

  # Default values for hyper-parameters we're going to sweep over
  config_defaults =  {
      'no_filters': 32,
      'filter_organization': 'same',
      'data_augmented': True, 
      'dropout' : 0.2,
      'batch_normalization': True,
  }

  # Initialize a new wandb run
  run = wandb.init(config=config_defaults, reinit=True)
  
  # Config is a variable that holds and saves hyperparameters and inputs
  config = wandb.config

  wandb.run.name = f'nf_{config.no_filters}_fo_{config.filter_organization}_dr_{config.dropout}\
                    _da_{config.data_augmented}_bn_{config.batch_normalization}'
  wandb.run.save()

  # Sweep uses L2 regularisation as default as given in the question
  modelA = CNN(inp_img_shape_1, f'{data_path}/train', get_klist(config.filter_organization, config.no_filters), F_list_1,
               no_neurons_dense_1, config_1, no_classes_1, activation_fn_list_1, P_list_1, S_list_1, config.batch_normalization, 
               config.dropout, f'{data_path}/val', f'{data_path}/test', config.data_augmented)
  run.finish()

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="abisheks", project="assignment2")
wandb.agent(sweep_id, lambda : sweep_wrapper())

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: mghk051h
Sweep URL: https://wandb.ai/abisheks/assignment2/sweeps/mghk051h


wandb: Agent Starting Run: q84iklqp with config:
wandb: 	batch_normalization: False
wandb: 	data_augmented: False
wandb: 	dropout: 0.3
wandb: 	filter_organization: double
wandb: 	no_filters: 32


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Found 9006 files belonging to 10 classes.
Found 1004 files belonging to 10 classes.
Found 2008 files belonging to 10 classes.
Epoch 1/20
141/141 - 77s - loss: 4.2053 - accuracy: 0.0940 - val_loss: 2.3028 - val_accuracy: 0.0996
Epoch 2/20
141/141 - 67s - loss: 2.3035 - accuracy: 0.0929 - val_loss: 2.3027 - val_accuracy: 0.0996
Epoch 3/20
141/141 - 66s - loss: 2.3034 - accuracy: 0.0869 - val_loss: 2.3027 - val_accuracy: 0.0996
Epoch 4/20
141/141 - 66s - loss: 2.3034 - accuracy: 0.0918 - val_loss: 2.3028 - val_accuracy: 0.0996
Epoch 5/20
141/141 - 66s - loss: 2.3033 - accuracy: 0.0927 - val_loss: 2.3027 - val_accuracy: 0.0996
Epoch 6/20
141/141 - 66s - loss: 2.3034 - accuracy: 0.0908 - val_loss: 2.3027 - val_accuracy: 0.0996
Epoch 7/20
141/141 - 66s - loss: 2.3033 - accuracy: 0.0920 - val_loss: 2.3028 - val_accuracy: 0.0996
Epoch 8/20
141/141 - 66s - loss: 2.3033 - accuracy: 0.0926 - val_loss: 2.3028 - val_accuracy: 0.0996
Epoch 9/20
141/141 - 66s - loss: 2.3034 - accuracy: 0.0912 - val_l

:W&B� �QT � �����
prjlufzgabisheksassignment2"�

batch_normalization�true

data_augmented�true

dropout�0.2
 
filter_organization�"double"


no_filters�32

_wandb�{}j810bdbbaadff������
("3.7.10*0.10.24B(� d0bc65748307403687c37b36b0f4da12~�(� 2

wandb-metadata.jsonr���
prjlufzgabisheksassignment2"�

batch_normalization�true

data_augmented�true

dropout�0.2
 
filter_organization�"double"


no_filters�32

_wandb�{}Bnf_32_fo_double_dr_0.2j810bdbbaadff������
("3.7.10*0.10.24B(g�	 2

model-best.h5�ekb� "���������Found 9006 files belonging to 10 classes.
Found 1004 files belonging to 10 classes.
Found 2008 files belonging to 10 classes.
Epoch 1/20
T�o� "����������wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
��ŭ- Z+
((@"3.7.10*0.10.24B(��� �}��>�